In [15]:
from neo4j.v1 import GraphDatabase

host = 'ec2-54-67-15-68.us-west-1.compute.amazonaws.com'
port = '9232'

uri = "bolt://" + host + ':' + port

driver = GraphDatabase.driver(uri, auth=("neo4j", "password"), encrypted=False)

session = driver.session()

# Delete the existing index on City nodes. This needs to happen in a separate transaction
try:
  with session.begin_transaction() as tx:
      tx.run('DROP INDEX ON :City(name)')
except Exception:
  print('ignored exception in drop index')

# Create the zip/city/county/state nodes and relationships and then query the data.
with session.begin_transaction() as tx:
    # Delete the existing nodes
    tx.run('MATCH (a:City{name:{city_name}}),(b:Zip{zipcode:{zipcode}}),(c:County{name: {county_name}}),(d:State{name:{state_name}}) DETACH DELETE a,b,c,d', zipcode='94518', city_name='Walnut Creek', county_name='Contra Costa', state_name='California')
    # Create new zip, city, county, and state nodes
    for record in tx.run('CREATE (a:City{name:{city_name},population:{population}}),(b:Zip{zipcode:{zipcode}}),(c:County{name: {county_name}}),(d:State{name:{state_name}}) return a,b,c,d', zipcode='94518', city_name='Walnut Creek', county_name='Contra Costa', state_name='California', population=70000):
        print(record)
    # Create the relationships between these nodes
    tx.run('MATCH (city:City{name:{city_name}}),(zip:Zip{zipcode:{zipcode}}),(county:County{name: {county_name}}),(state:State{name:{state_name}}) create (zip)-[:in_city]->(city)-[:in_county]->(county)-[:in_state]->(state)', zipcode='94518', city_name='Walnut Creek', county_name='Contra Costa', state_name='California')
    # Query the zip, city, county and state where the city.name is Walnut Creek
    for record in tx.run('match (zip:Zip)-[:in_city]->(city:City{name:{city_name}})-[:in_county]->(county:County)-[:in_state]->(state:State) return zip.zipcode,county.name,state.name;', city_name='Walnut Creek'):
        print(record)
    # Query the zip, city, county and state objects where the city.name is Walnut Creek
    for record in tx.run('match (zip:Zip)-[:in_city]->(city:City{name:{city_name}})-[:in_county]->(county:County)-[:in_state]->(state:State) return city, zip,county,state;', city_name='Walnut Creek'):
        print(record)
    # Query the zip, city, county and state objects where the state.name is California
    for record in tx.run('match (zip:Zip)-[:in_city]->(city:City)-[:in_county]->(county:County)-[:in_state]->(state:State{name:{state_name}}) return city, zip,county,state;', state_name='California'):
        print(record)

with session.begin_transaction() as tx:
    tx.run('CREATE INDEX ON :City(name)')

<Record a=<Node id=457046 labels={'City'} properties={'name': 'Walnut Creek', 'population': 70000}> b=<Node id=457049 labels={'Zip'} properties={'zipcode': '94518'}> c=<Node id=457002 labels={'County'} properties={'name': 'Contra Costa'}> d=<Node id=457003 labels={'State'} properties={'name': 'California'}>>
<Record zip.zipcode='94518' county.name='Contra Costa' state.name='California'>
<Record city=<Node id=457046 labels={'City'} properties={'name': 'Walnut Creek', 'population': 70000}> zip=<Node id=457049 labels={'Zip'} properties={'zipcode': '94518'}> county=<Node id=457002 labels={'County'} properties={'name': 'Contra Costa'}> state=<Node id=457003 labels={'State'} properties={'name': 'California'}>>
<Record city=<Node id=457046 labels={'City'} properties={'name': 'Walnut Creek', 'population': 70000}> zip=<Node id=457049 labels={'Zip'} properties={'zipcode': '94518'}> county=<Node id=457002 labels={'County'} properties={'name': 'Contra Costa'}> state=<Node id=457003 labels={'State'